In [6]:
import os
import numpy as np
import pandas as pd

import torch

import datasets
from transformers import AutoTokenizer, AutoModelForCausalLM
from backend_utils import _setHFToken, spiltDataset
from backend_utils import quantization_config, model_id

In [7]:
# temp = os.listdir("../")
# print(temp)

if torch.cuda.is_available():
    device=torch.device(type='cuda',index=0)
else:
    device=torch.device(type='cpu',index=0)
print(type(device))

<class 'torch.device'>


In [8]:
# import torch; assert torch.cuda.get_device_capability()[0] >= 8, 'Hardware not supported for Flash Attention'
os.environ['HF_TOKEN'] = _setHFToken()

In [9]:
dataset = datasets.load_from_disk("../dataset")
print(dataset)
dataset = spiltDataset(dataset)
print(dataset)

Dataset({
    features: ['id', 'messages'],
    num_rows: 71878
})
DatasetDict({
    train: Dataset({
        features: ['id', 'messages'],
        num_rows: 57502
    })
    test: Dataset({
        features: ['id', 'messages'],
        num_rows: 7187
    })
    valid: Dataset({
        features: ['id', 'messages'],
        num_rows: 7189
    })
})


In [10]:
print(dataset['train'][0]['messages'])

[{'content': "You are a helpful and joyous mental therapy assistant. Always answer as helpfully and cheerfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.", 'role': 'system'}, {'content': 'hey i have something exciting to share with you today. i have recently discovered a new hobby that has brought me so much joy and fulfillment!', 'role': 'human'}, {'content': 'that s wonderful ! i am thrilled to hear that you ve found something that excites you. tell me more about this new hobby. what led you to explore it?', 'role': 'assistant'}, {'content': 'well it all started when my friend introduced me to a new gro

In [11]:
print(f"[INFO] Using model_id: {model_id}")
print(quantization_config)
output_dir = "../results"

[INFO] Using model_id: google/gemma-7b
BitsAndBytesConfig {
  "_load_in_4bit": true,
  "_load_in_8bit": false,
  "bnb_4bit_compute_dtype": "float16",
  "bnb_4bit_quant_storage": "uint8",
  "bnb_4bit_quant_type": "nf4",
  "bnb_4bit_use_double_quant": false,
  "llm_int8_enable_fp32_cpu_offload": false,
  "llm_int8_has_fp16_weight": false,
  "llm_int8_skip_modules": null,
  "llm_int8_threshold": 6.0,
  "load_in_4bit": true,
  "load_in_8bit": false,
  "quant_method": "bitsandbytes"
}



In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id, token = os.environ['HF_TOKEN'])
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map = device,
    token = os.environ['HF_TOKEN']
)
model.to(device)